In [ ]:
import geopandas as gpd
import contextily as cx
import matplotlib.pyplot as plt
import shapely.geometry as sg
import pathlib
import shutil
from tqdm.notebook import tqdm

import set_path
from gvl.helper_functions import (roundup, rounddown, box_to_name,
                                  get_tilecode_from_filename)

In [ ]:
dataset_path = '../datasets/validation/'

RD_CRS = 'epsg:28992'
LL_CRS = 'WGS84'

In [ ]:
area = gpd.read_file(f'{dataset_path}correct_trees_areas_oosterpark.shp', crs=RD_CRS)
trees = gpd.read_file(f'{dataset_path}correct_trees_oosterpark.shp', crs=LL_CRS)

In [ ]:
trees.to_crs(RD_CRS, inplace=True)

In [ ]:
%matplotlib widget

fig, ax = plt.subplots(1, figsize=(8,5))

area.plot(ax=ax, color='lightblue', alpha=0.8)
trees.plot(ax=ax, color='darkgreen', marker='.', markersize=5, alpha=1)
cx.add_basemap(ax=ax, crs=RD_CRS)

In [ ]:
# Get bounds of city area
bounds = area.unary_union.bounds

tile_size = 1000

In [ ]:
BASE_FOLDER = '/media/dbloembergen/PointCloud/AHN4'
ahn_subtile_folder = f'{BASE_FOLDER}/AMS_subtiles_1000/'

In [ ]:
subtiles = list(pathlib.Path(ahn_subtile_folder).glob('*.laz'))
subtiles_codes = [get_tilecode_from_filename(subtile.name) for subtile in subtiles]

In [ ]:
def code_to_geom(code):
    parts = code.split('_')
    x, y = int(parts[0])*tile_size, int(parts[1])*tile_size
    return sg.box(x, y, x+tile_size, y+tile_size)

geoms = [code_to_geom(code) for code in subtiles_codes]

In [ ]:
subtiles_gdf = gpd.GeoDataFrame({'subtile': subtiles,
                                 'name': subtiles_codes,
                                 'geometry': geoms}, crs=RD_CRS)

In [ ]:
subtiles_gdf = subtiles_gdf[subtiles_gdf.intersects(area.unary_union)]

In [ ]:
subtiles_gdf

In [ ]:
subtiles_gdf.name.values

In [ ]:
out_folder = '../datasets/AHN4/AMS_subtiles_1000/'
dry_run = True

laz_files = subtiles_gdf.subtile.values
npz_files = [f'{BASE_FOLDER}/npz_subtiles_1000/ahn_{name}.npz' for name in subtiles_gdf.name.values]

for f in tqdm(laz_files):
    if dry_run:
        print(f'{f.as_posix()} => {out_folder}{f.name}')
    else:
        shutil.copy(f.as_posix(), f'{out_folder}{f.name}')